In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df=pd.read_pickle("dataset/cic_processed.pkl")
print(df.head())

        Protocol  Flow Duration  Tot Fwd Pkts  Tot Bwd Pkts  TotLen Fwd Pkts  \
453520         6           1207             5             2              935   
339493         6           1151             5             2              935   
739247         6           1348             2             0                0   
608461         6           1302             5             2              935   
881208         6          15181             5             2              935   

        TotLen Bwd Pkts  Fwd Pkt Len Max  Fwd Pkt Len Min  Fwd Pkt Len Mean  \
453520              314              935                0             187.0   
339493              287              935                0             187.0   
739247                0                0                0               0.0   
608461              325              935                0             187.0   
881208              309              935                0             187.0   

        Fwd Pkt Len Std  ...  Fwd Seg Size M

In [4]:
X=df.drop(columns=['Label'])
Y=df['Label']
original_features=X.columns
print(X.shape,Y.shape)

(425775, 77) (425775,)


In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42,stratify=Y)

In [6]:
from sklearn.feature_selection import SelectKBest, f_classif #feature selection

selector=SelectKBest(score_func=f_classif,k=20)
X_train=selector.fit_transform(X_train,Y_train)
X_test=selector.transform(X_test)
selected_cols=original_features[selector.get_support()]
print(selected_cols)

Index(['Fwd Pkt Len Max', 'Fwd Pkt Len Mean', 'Fwd Pkt Len Std',
       'Bwd Pkt Len Mean', 'Flow Byts/s', 'Pkt Len Max', 'Pkt Len Mean',
       'Pkt Len Std', 'Pkt Len Var', 'RST Flag Cnt', 'PSH Flag Cnt',
       'ACK Flag Cnt', 'ECE Flag Cnt', 'Down/Up Ratio', 'Pkt Size Avg',
       'Fwd Seg Size Avg', 'Bwd Seg Size Avg', 'Subflow Bwd Pkts',
       'Init Fwd Win Byts', 'Init Bwd Win Byts'],
      dtype='object')


C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [31 32 33 49 55 56 57 58 59 60] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\hp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw


In [7]:
from sklearn.ensemble import RandomForestClassifier 
model= RandomForestClassifier(n_estimators=120,n_jobs=-1,min_samples_leaf=3,max_features='sqrt',random_state=42)
model.fit(X_train,Y_train)

RandomForestClassifier(min_samples_leaf=3, n_estimators=120, n_jobs=-1,
                       random_state=42)

In [9]:
from sklearn.metrics import confusion_matrix,classification_report
#Y_pred=model.predict(X_test)
Y_pred_prob=model.predict_proba(X_test)[:,1]
#cm=confusion_matrix(Y_test,Y_pred)
#print(cm)
#print(classification_report(Y_test,Y_pred))
print(Y_pred_prob[:20])

[0.99999865 0.         0.         0.         0.         0.
 0.99984479 0.99984479 0.         0.         0.         0.00363745
 0.         0.         0.         0.         0.99984479 0.
 0.         0.        ]


In [11]:
importances = model.feature_importances_

importance_df = pd.DataFrame({
    'Feature': selected_cols,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(importance_df.head(15))

              Feature  Importance
0     Fwd Pkt Len Max    0.232502
1    Fwd Pkt Len Mean    0.159055
15   Fwd Seg Size Avg    0.155669
19  Init Bwd Win Byts    0.123089
2     Fwd Pkt Len Std    0.085898
11       ACK Flag Cnt    0.055862
6        Pkt Len Mean    0.036842
10       PSH Flag Cnt    0.029170
14       Pkt Size Avg    0.026012
16   Bwd Seg Size Avg    0.024375
5         Pkt Len Max    0.024299
4         Flow Byts/s    0.011920
18  Init Fwd Win Byts    0.009724
7         Pkt Len Std    0.008267
3    Bwd Pkt Len Mean    0.006770


In [10]:
tn,fp,fn,tp=cm.ravel()
far=fp/(fp+tn)
print(far)

NameError: name 'cm' is not defined